In [1]:
from google.cloud import bigquery
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import datetime 
import requests
import json
import tensorflow as tf
import plotly.graph_objects as go
import plotly.express as px

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt
# nvcc --version  ###CUDA version

import pathlib
import shutil
from numpy.random import seed
seed(2)
tf.random.set_seed(2)
from math import sqrt
import os

random_state = 42
# import mlflow
# exp_id = 'weather_dataset'
# mlflow.set_experiment(exp_id)
# mlflow.set_tracking_uri('file:///C:/Users/gabri/VSCode%20Projects/Weather%20Prediction/mlruns')
# mlflow.autolog()
# mlflow.log_param('random_state', random_state)
# export MLFLOW_TRACKING_URI=http://192.168.0.1:5000


In [2]:
# 1) define training, testing and prediction daterange 
# 2) define parameters, feature engineering and transformations, create dataframe 
# 3) convert to numpy and reshape 
# 4) define train, test and val split
# 5) normalize
# 6) define model and log in MLFlow
# 7) model training

In [6]:
# project details and gcp connection
project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
key_path = r'C:\Users\gabri\GCP\cohesive-geode-404308-1242b1e12c66.json'
key_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')
client = bigquery.Client.from_service_account_json(key_path)

In [32]:
# get data query
# training_start = '2021-08-01'
# training_end = '2021-08-12'
training_start = '2021-08-01 00:00:00'  # Specify the start time with hours, minutes, and seconds
training_end = '2022-08-11 23:01:00'  # Specify the end time with hours, minutes, and seconds

location_1 = 'City of London'
location_2 = 'Cockfosters'

# date_obj_start = datetime.datetime.strptime(training_start, '%Y-%m-%d')
# date_obj_end = datetime.datetime.strptime(training_end, '%Y-%m-%d')
date_obj_start = datetime.datetime.strptime(training_start, '%Y-%m-%d %H:%M:%S')  # Include '%H:%M:%S'
date_obj_end = datetime.datetime.strptime(training_end, '%Y-%m-%d %H:%M:%S')  # Include '%H:%M:%S'
unix_start = int(date_obj_start.timestamp())
unix_end = int(date_obj_end.timestamp())


query = f"""
select dt, temp - 273.15 as temp, pressure, humidity
from {project_id}.weather_training.weather_city_of_london_training
where dt > {unix_start} and dt < {unix_end}
order by dt
"""
query_job = client.query(query)
df_filtered_cl = query_job.to_dataframe()

query = f"""
select dt, temp - 273.15 as temp, pressure, humidity
from {project_id}.weather_training.weather_cockfosters_historical
where dt > {unix_start} and dt < {unix_end}
order by dt
"""
query_job = client.query(query)
df_filtered_cf = query_job.to_dataframe()
print(f'Gets data from {training_start} to {training_end} from {location_1} and {location_2} and merges the two tables')

Gets data from 2021-08-01 00:00:00 to 2022-08-11 23:01:00 from City of London and Cockfosters and merges the two tables


In [48]:
query = f"""
SELECT
  cod,
  name,
  dt,
  clouds.*,
  main.*,
  coord.*
FROM `{project_id}.weather_api.weather_api_table`;
"""
query_job = client.query(query)
df = query_job.to_dataframe()
df

,cod,name,dt,all,pressure,temp,temp_max,temp_min,humidity,feels_like,lat,lon
0,200,Hadley Wood,1699896297,14,1011,285.55,286.22,284.47,75,284.80,51.6479,-0.1513
1,200,Hadley Wood,1699898278,15,1012,285.49,286.20,284.47,76,284.76,51.6479,-0.1513
2,200,Hadley Wood,1699898278,15,1012,285.49,286.20,284.47,76,284.76,51.6479,-0.1513
3,200,Hadley Wood,1700070556,23,1030,281.15,282.73,279.35,81,281.15,51.6479,-0.1513
4,200,Hadley Wood,1700070556,23,1030,281.15,282.73,279.35,81,281.15,51.6479,-0.1513
...,...,...,...,...,...,...,...,...,...,...,...,...
113,200,Hadley Wood,1700208006,5,1031,276.53,278.59,274.35,95,273.50,51.6479,-0.1513
114,200,Hadley Wood,1700215206,6,1032,279.37,281.11,277.79,94,279.37,51.6479,-0.1513
115,200,Hadley Wood,1700218804,6,1033,281.21,283.16,279.18,87,281.21,51.6479,-0.1513
116,200,Hadley Wood,1700222404,8,1033,282.91,284.50,280.67,78,282.91,51.6479,-0.1513


In [ ]:
# data 

In [ ]:
# rename columns to distinguish them by location, each location is treated as a different feature
df_filtered_cl.rename(columns={'dt': 'dt_cl', 'temp': 'cl_temp', 'pressure': 'cl_pressure', 'humidity': 'cl_humidity'}, inplace=True)
df_filtered_cf.rename(columns={'dt': 'dt_cf', 'temp': 'cf_temp', 'pressure': 'cf_pressure', 'humidity': 'cf_humidity'}, inplace=True)

# merge tables / dataframes
merged_df = pd.merge(df_filtered_cl, df_filtered_cf, how='left', left_on='dt_cl', right_on='dt_cf')

# copy dataframe

# filter and copy dataframe columns, select one timestamp and rename 
columns_to_keep = ['dt_cl', 'cl_temp']
features = ['cl_temp']
final_df = merged_df[columns_to_keep].copy()
final_df.rename(columns={'dt_cl': 'dt'}, inplace=True)

print(f"Timestamp column name changed to 'dt'")
print(f'Features selected: {columns_to_keep}')
print(final_df.head(1))

In [ ]:
# create arrays from dataframe columns, convert Unix timestamp to UTC datetime

index_list=[]
timestamp_list=[]
for idx, timestamp in enumerate(final_df['dt'].to_numpy()):
    index_list.append(idx) 
    timestamp_list.append(timestamp)

datetime_list = [datetime.datetime.utcfromtimestamp(event) for event in timestamp_list]
final_df = final_df.drop(columns='dt', axis=0)
dataset_length = len(index_list)
print(f'Number of indices: {dataset_length}')
print('Converted dataframe columns to lists')

In [ ]:
start_date = datetime.datetime.utcfromtimestamp(timestamp_list[0])
end_date = datetime.datetime.utcfromtimestamp(timestamp_list[-1])

context_hours = 24
prediction_length = 24
eliminated_timesteps = context_hours - 1

print(f'Dataset starts on {start_date}')
print(f'Dataset ends on {end_date}')
print(f'Original length of dataset: {dataset_length} hours')


print(f'Length of context: {context_hours} hours')
print(f'Number of hours to predict predict: {prediction_length} hours')
print(f'Given temperature from the last {context_hours} hours, we want to predict temperature over the next {prediction_length} hours')
print(f'In order to create windows, we lose {eliminated_timesteps} hours to create the first window')

new_dataset_length = dataset_length - eliminated_timesteps
window_count = new_dataset_length
print(f'Number of datapoints for training, validation and testing: {new_dataset_length} hours')

In [ ]:
# 7 Define train, val and test ratios
print(f'Number of indices available to create window, train, validate and test: {dataset_length}')
train = 0.8
val = 0.2
window_start = 0
print(f'Train and validation split: {train} - {val}')
print(f'Eliminated datapoints: 0 - {eliminated_timesteps}')
train_start = eliminated_timesteps
val_start = int((dataset_length-prediction_length)*train)
print(f'Train: {train_start}-{val_start}')
test_start = dataset_length-prediction_length
print(f'Validate: {val_start}-{test_start}')
print(f'Test: {val_start} - {dataset_length}')

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="The behavior of DatetimeProperties.to_pydatetime is deprecated*")

fig = go.Figure()
window_trace = px.scatter(x=datetime_list[window_start:train_start], y=final_df['cl_temp'].iloc[window_start:train_start]).data[0]
window_trace.update(line=dict(color='purple'), marker=dict(color='purple'), name=f'Eliminated indices: {window_start}-{train_start}. Count:{train_start-window_start}', showlegend=True)

# Plot the training data
train_trace = px.scatter(x=datetime_list[train_start:val_start], y=final_df['cl_temp'][train_start:val_start]).data[0]
train_trace.update(line=dict(color='blue'), marker=dict(color='blue'), name=f'Train Data: {train_start}-{val_start}. Count:{val_start-train_start}', showlegend=True)

# Plot the validation data
val_trace = px.scatter(x=datetime_list[val_start:test_start], y=final_df['cl_temp'][val_start:test_start]).data[0]
val_trace.update(line=dict(color='green'), marker=dict(color='green'), name=f'Validation Data: {val_start}-{test_start}. Count:{test_start-val_start}', showlegend=True)

# Plot the test data
test_trace = px.scatter(x=datetime_list[test_start: dataset_length], y=final_df['cl_temp'][test_start: dataset_length]).data[0]
test_trace.update(line=dict(color='red'), marker=dict(color='red'), name=f'Test Data: {test_start}-{dataset_length}. Count:{dataset_length-test_start}', showlegend=True)

# Plot all data
all_trace = px.scatter(x=datetime_list, y=final_df['cl_temp'].to_list()).data[0]
all_trace.update(line=dict(color='yellow'), marker=dict(color='yellow'), name=f'All Data', showlegend=True, mode='lines')

fig.add_trace(window_trace)
fig.add_trace(train_trace)
fig.add_trace(val_trace)
fig.add_trace(test_trace)
fig.add_trace(all_trace)

fig.update_layout(showlegend=True)
fig.show()

In [ ]:
# Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(final_df.iloc[train_start:val_start])
df_scaled = pd.DataFrame(scaler.transform(final_df))
df_scaled.columns = final_df.columns
print(f'We calculate the mean and std of the training data and use this to normalise the entire dataframe')
print(final_df.head(1))
print(df_scaled.head(1))

In [ ]:
total_length = len(df_scaled['cl_temp'])
data = df_scaled['cl_temp'].to_numpy()
X = []
y = []

for i in range(total_length - context_hours):
    X.append(data[i:i + context_hours])
    y.append(data[i + context_hours:i + context_hours + 1]) 

X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
# # import mlflow
# import numpy as np
# from tensorflow import keras
# from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, InputLayer

# feature_count = len(features)
# model = keras.Sequential()

# model.add(InputLayer((context_hours, feature_count)))
# model.add(Bidirectional(LSTM(64, activation='tanh')))
# model.add(Dense(256, 'relu'))
# model.add(Dense(64, 'relu'))
# model.add(Dropout(0.25))
# model.add(Dense(feature_count, 'linear'))

# model.summary()

In [ ]:
# from keras.metrics import RootMeanSquaredError
# from keras.optimizers import Adam

# learning_rate = 0.0005
# epochs = 8
# loss = 'mse'
# batch_size = 128
# model.compile(loss=loss, optimizer=Adam(learning_rate = learning_rate), metrics=[RootMeanSquaredError()])
# history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=False)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])

In [ ]:
# # scale and get mae
# y_pred = model.predict(X_test)
# y_pred = scaler.inverse_transform(y_pred)
y_test_final = scaler.inverse_transform(y_test)
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# print(f'MAE: {round(mean_absolute_error(y_test_final, y_pred), 2)}')
# print(f'RMSE: {round(sqrt(mean_squared_error(y_test_final, y_pred)), 2)}')

In [ ]:
# plt.scatter(x=[i for i in range(len(y_pred))], y=y_test_final.reshape(len(y_test_final),) )
# plt.scatter(x=[i for i in range(len(y_pred))], y=y_pred.reshape(len(y_pred),) )

In [ ]:
# model.save('cf_cl.h5')
# model.save('test_model.keras')
# # loaded_model = tf.keras.models.load_model('heath.h5')
# loaded_model

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('test_model.keras')

In [ ]:
loaded_model.save('test_model.h5')


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("test_model_lite")
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
input_temperature = X_test[0]
true_temperature = y_test_final[0]
predicted_temperature_scaled = loaded_model.predict(input_temperature.reshape(1, input_temperature.shape[0]))
predicted_temperature = scaler.inverse_transform(predicted_temperature_scaled)

print('True temperature: ', true_temperature)
print('predicted_temperature: ', predicted_temperature)
print('Input data: ', scaler.inverse_transform(input_temperature.reshape(input_temperature.shape[0], 1)))